実装にあたってこちらの記事を参考にさせていただきました  
[Zenn 歩行時の加速度,角速度データから歩行軌跡を描いてみた - Beyond Toyama](https://zenn.dev/happy663/articles/1a0abc285b3a33)

# 事前準備

## ライブラリのインポート

In [131]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [132]:
import time
import requests
import random

# API実行時間計測

In [138]:
url = "http://localhost:8082/api/v1/stayers"
baseURL = "http://localhost:8082"

# ヘッダー
headers = {
    "Content-Type": "application/json"
}

## ユーザ登録（APIを用いて）

In [ ]:
def createBasicBeaconUsers(numUsers):
  for i in range(numUsers):
    name = ''.join(random.choices('abcdefghijklmnopqrstu', k=7))
    uuid = ''.join(random.choices('0123456789abcdef', k=5))
    data = {
        "name":name,
        "uuid":uuid,
        "email":"",
        "role":1,
        "communityId":1,
        "beaconName":"FCS1301",
        "tagIds":[1,5,10]
    }
    response = requests.post(baseURL+"/api/v1/users", json=data, headers=headers)
    print(response.json())

{'status': 'ok'}
{'status': 'ok'}


## リクエストを定義

### エンドポイントとヘッダーの設定

### ボディデータを設定する関数を定義

In [125]:
# 従来手法の未登録のユーザをリクエストに含める
def getRequestBasicNonRegisterdUser(numUsers):
    data = {
        "beacons": [],
        "roomId": 1
    }
    for i in range(numUsers):
        hexString = ''.join(random.choices('0123456789abcdef', k=32))
        # ボディデータ
        data["beacons"].append({
            "uuid": hexString,
            "rssi": -49
        })
    print(data)
    return data

# 従来手法の登録済みのユーザをリクエストに含める
def getRequestBasicRegisterdUser(numUsers):
    # ボディデータ
    data = {
        "beacons": [],
        "roomId": 1
    }
    for i in range(numUsers):
        hexString = ''.join(random.choices('0123456789abcdef', k=32))
        # ボディデータ
        data["beacons"].append({
            "uuid": hexString,
            "rssi": -49
        })
    print(data)
    return data
    # data = {
    #     "beacons": [
    #         {
    #             "uuid": "4c000180000021000022000121000024000023",
    #             "rssi": -49
    #         },
    #         {
    #             "uuid": "8ebc21144abdba0db7c6ff0a00200002",
    #             "rssi": -49
    #         },
    #         {
    #             "uuid":"e7d61ea3f8dd49c88f2ff2484c07ac02",
    #             "rssi": -50
    #         },
    #         {
    #             "uuid":"e7d61ea3f8dd49c88f2ff2484c07ac02",
    #             "rssi": -50
    #         },
    #         {
    #             "uuid":"e7d61ea3f8dd49c88f2ff2484c07ac02",
    #             "rssi": -50
    #         }
    #     ],
    #     "roomId": 1
    # }
    # return data

## 計測

### 従来手法での計測

In [ ]:
# file_path = 'data/basic.csv'
# # CSVファイルを読み込む
# df = pd.read_csv(file_path_advertise)

In [ ]:
print("start bench")

for i in range(100):
  print(str(i) + "/100回目")
  # 100人ランダムな従来ユーザを登録する（PythonでWebAPIを用いて）
  createBasicBeaconUsers(2)
  # ランダムなユーザ(登録済み)を10人POSTで投げる
  start = time.perf_counter()
  # response = requests.post(url, json=getRequestBasicNonRegisterdUser(10), headers=headers)
  end = time.perf_counter()
  # ランダムなユーザ(未登録)を10人POSTで投げる
  start = time.perf_counter()
  response = requests.post(url, json=getRequestBasicNonRegisterdUser(10), headers=headers)
  end = time.perf_counter()


start = time.perf_counter()

response = requests.post(url, json=getRequestBasicNonRegisterdUser(10), headers=headers)

end = time.perf_counter()
print(response.json())
print("end bench")
print("実行時間: " + str(end - start) + " sec")

start bench
{'beacons': [{'uuid': 'ef532d5edf292965ac5d0ad2328cb5f5', 'rssi': -49}, {'uuid': 'e03177241e94ec72a9bf093bad58085e', 'rssi': -49}, {'uuid': '310b976967edfad39a8a1540b4097e0f', 'rssi': -49}, {'uuid': '52158990c2996e31bccd64aee350700a', 'rssi': -49}, {'uuid': '689d93b8938af9ce8dba09b2e6b5eab7', 'rssi': -49}, {'uuid': 'd046512194ca9bc9f0fd9a43def6e139', 'rssi': -49}, {'uuid': 'c9ebc4205f501fc5ded60007497952fd', 'rssi': -49}, {'uuid': 'b43743f0f48622fac7954708d45494b2', 'rssi': -49}, {'uuid': '5c509bd5d9b5324698a7415c38beb7dd', 'rssi': -49}, {'uuid': 'd49e177ed3895ecf499049e84820015a', 'rssi': -49}], 'roomId': 1}
{'status': 'OK'}
end bench
実行時間: 0.07589337500030524 sec


### 滞在情報をリセット

In [96]:
# ボディデータ
data = {
    "beacons": [
    ],
    "roomId": 1
}
response = requests.post(url, json=data, headers=headers)